In [8]:
from torch.utils.data import DataLoader,IterableDataset
from datasets import load_dataset

from transformers import AutoModel, AutoTokenizer
import torch
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
from torch import nn
from copy import deepcopy
import numpy as np
import gc

from tqdm.auto import tqdm
%load_ext autoreload
%autoreload 2
    

import sys

src_path = os.path.abspath(os.path.join('..', 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)

from utils import convert_to_Qwen2_ND, detect_domain_specific_neurons,impacts_on, Collator, get_all_dsn

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
qwen_path = "D:\models\Qwen2.5-1.5B"

qwen = AutoModel.from_pretrained(qwen_path, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(qwen_path)

In [3]:
qwen = convert_to_Qwen2_ND(qwen).to("cuda")
qwen.eval();

In [4]:
collator = Collator(tokenizer)
dataset = load_dataset('yhavinga/ccmatrix', "en-ru", split='train', streaming=True, trust_remote_code=True)
data_loader = DataLoader(
    dataset,
    batch_size=2,
    shuffle=False,
    collate_fn = collator,
    batch_sampler=None 
)

In [5]:
# for elems in data_loader:
#     print(elems)
#     break

In [6]:
qwen, output = detect_domain_specific_neurons(
    qwen,
    tokenizer,
    dataloader=data_loader,
    eps=1e-1,
    domain_name="eng", 
    reset_impacts=False,
    reset_dsn=True,
    num_elements=100
)

  0%|          | 0/100 [00:00<?, ?it/s]

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
d:\Desktop\9сем\CreationOfIntelligentSystems_Multilingual_Interpretability\src\modules.py:183: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [10]:
all_dsn = get_all_dsn(qwen)

In [11]:
all_dsn[0]

{'mlp': {'eng': {'up_proj': tensor([ True, False, False,  ..., False,  True, False], device='cuda:0'),
   'down_proj': tensor([ True, False, False,  ..., False,  True, False], device='cuda:0')}},
 'attn': {'eng': {'q_proj': tensor([[False, False, False,  ...,  True,  True,  True],
           [False, False, False,  ..., False, False, False],
           [False, False,  True,  ...,  True,  True,  True],
           ...,
           [False, False, False,  ...,  True,  True,  True],
           [False, False, False,  ..., False, False, False],
           [ True,  True,  True,  ..., False, False, False]], device='cuda:0'),
   'k_proj': tensor([[False, False, False,  ...,  True,  True,  True],
           [False, False, False,  ..., False, False, False],
           [False, False,  True,  ...,  True,  True,  True],
           ...,
           [False, False, False,  ...,  True,  True,  True],
           [False, False, False,  ..., False, False, False],
           [ True,  True,  True,  ..., False, F